In [2]:
include(dirname(dirname(pwd()))*"\\src\\TuLiPa.jl");
using .TuLiPa
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS

In [5]:
function make_connection(from, to)    
    flow_name = "$(from)$(to)"
    arrow_name_from = "arrow_from$(from)$(to)"
    arrow_name_to = "arrow_to$(from)$(to)"
    balance_from = "$(from)"
    balance_to = "$(to)"
    cap_name = "$(flow_name)_cap"
    elem = [
        DataElement("Flow", "BaseFlow", flow_name, Dict()),    
        DataElement("Arrow", "BaseArrow", arrow_name_from, Dict("Balance" => balance_from, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "Out")),
        DataElement("Arrow", "BaseArrow", arrow_name_to, Dict("Balance" => balance_to, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "In")),
        #DataElement("Capacity", "PositiveCapacity", cap_name, Dict("Param" => cap_name, "WhichInstance" => flow_name, "WhichConcept" => "Flow", "Bound" => "Upper")), 
        #DataElement("Param", "MWToGWhSeriesParam", cap_name, Dict{Any, Any}("Level" => BC_cap, "Profile" => 1.0)),
    ]    
    return elem
end

function make_flowbased(ptdfs)
    elem = Array{DataElement}([])
    for row in eachrow(ptdfs)
        line = row["line"] # Flow variable used for transfer, i.e AB        
        ptdfs_names = names(ptdfs)
        ptdfs_val = Array(row)    
        ptdfs_val, ptdfs_names, max_cap = TuLiPa.get_values_from_ptdf_df(ptdfs_val, ptdfs_names)        
        e = DataElement("FlowBased", "BaseFlowBased", line, Dict(
                "Flow" => line,
                "ptdfs_names" => ptdfs_names,
                "ptdfs" => ptdfs_val,
                "max_cap" => max_cap)
            )
        push!(elem, e)
    end
    return elem
end


make_flowbased (generic function with 1 method)

In [6]:
PowerA = 1.0
PowerA_cap = 10000.0
DemandA = 10.0
PowerB_cap = 10000.0
PowerB = 10000.0
DemandB = 100.0
PowerC_cap = 10000.0
PowerC = 100.0
DemandC = 10.0

#AB_cap = 1000.0
#AC_cap = 2000.0
#BC_cap = 3000.0

elements = [
    
    DataElement("Balance", "BaseBalance", "A", Dict{Any, Any}("Commodity" => "Power")),    
    DataElement("Flow", "BaseFlow", "PowerA", Dict{Any, Any}()),
    DataElement("Arrow", "BaseArrow", "A", Dict{Any, Any}("Balance" => "A", "Flow" => "PowerA", "Conversion" => 1.0, "Direction" => "In")),
    DataElement("Cost", "CostTerm", "PowerA_cost", Dict{Any, Any}("Param" => PowerA, "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Direction" => "In")),
    DataElement("Capacity", "PositiveCapacity", "PowerA_cap", Dict{Any, Any}("Param" => "PowerA_cap", "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Bound" => "Upper")),
    DataElement("Param", "MWToGWhSeriesParam", "PowerA_cap", Dict{Any, Any}("Level" => PowerA_cap, "Profile" => 1.0)),
    DataElement("Param", "MWToGWhSeriesParam", "DemandA", Dict("Level" => DemandA , "Profile" => 1.0)),
    DataElement("RHSTerm", "BaseRHSTerm", "DemandA", Dict{Any, Any}("Balance" => "A", "Param" => "DemandA", "Direction" => "Out")),

    DataElement("Balance", "BaseBalance", "B", Dict{Any, Any}("Commodity" => "Power")),    
    DataElement("Flow", "BaseFlow", "PowerB", Dict{Any, Any}()),
    DataElement("Arrow", "BaseArrow", "B", Dict{Any, Any}("Balance" => "B", "Flow" => "PowerB", "Conversion" => 1.0, "Direction" => "In")),
    DataElement("Cost", "CostTerm", "PowerB_cost", Dict{Any, Any}("Param" => PowerB, "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Direction" => "In")),
    DataElement("Capacity", "PositiveCapacity", "PowerB_cap", Dict{Any, Any}("Param" => "PowerB_cap", "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Bound" => "Upper")),
    DataElement("Param", "MWToGWhSeriesParam", "PowerB_cap", Dict{Any, Any}("Level" => PowerB_cap, "Profile" => 1.0)),
    DataElement("Param", "MWToGWhSeriesParam", "DemandB", Dict("Level" => DemandB , "Profile" => 1.0)),
    DataElement("RHSTerm", "BaseRHSTerm", "DemandB", Dict{Any, Any}("Balance" => "B", "Param" => "DemandB", "Direction" => "Out")),

    DataElement("Balance", "BaseBalance", "C", Dict{Any, Any}("Commodity" => "Power")),    
    DataElement("Flow", "BaseFlow", "PowerC", Dict{Any, Any}()),
    DataElement("Arrow", "BaseArrow", "C", Dict{Any, Any}("Balance" => "C", "Flow" => "PowerC", "Conversion" => 1.0, "Direction" => "In")),
    DataElement("Cost", "CostTerm", "PowerC_cost", Dict{Any, Any}("Param" => PowerC, "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Direction" => "In")),
    DataElement("Capacity", "PositiveCapacity", "PowerC_cap", Dict{Any, Any}("Param" => "PowerC_cap", "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Bound" => "Upper")),
    DataElement("Param", "MWToGWhSeriesParam", "PowerC_cap", Dict{Any, Any}("Level" => PowerC_cap, "Profile" => 1.0)),
    DataElement("Param", "MWToGWhSeriesParam", "DemandC", Dict("Level" => DemandC , "Profile" => 1.0)),
    DataElement("RHSTerm", "BaseRHSTerm", "DemandC", Dict{Any, Any}("Balance" => "C", "Param" => "DemandC", "Direction" => "Out"))
]
    
elements = vcat(elements, make_connection("A", "B"))
elements = vcat(elements, make_connection("B", "A"))
            
elements = vcat(elements, make_connection("A", "C"))
elements = vcat(elements, make_connection("C", "A"))        
    
elements = vcat(elements, make_connection("B", "C"))
elements = vcat(elements, make_connection("C", "B"))

power_horizon = SequentialHorizon(1, Day(1))
push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", (HORIZON_CONCEPT, power_horizon)))
addscenariotimeperiod!(elements, "ScenarioTimePeriod", getisoyearstart(1981), getisoyearstart(1983));
        
ptdfs = DataFrame(
    line = ["AB", "AC", "BC"],
    max_cap = [0, 100, 100],
    AB = [0.0, 1.0, 1.0],
    AC = [0.0, 1.0, 0.0],
    BA = [1.0, 0.0, 0.0],
    BC = [0.0, 0.0, 1.0],
    CA = [0.0, 1.0, 0.0],
    CB = [0.0, 0.0, 1.0]
)

elements = vcat(elements,  make_flowbased(ptdfs))

    
modelobjects = getmodelobjects(elements)
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)
        
update!(prob, prob_time)
solve!(prob)
print(prob.model)
print(solution_summary(prob.model, verbose = true))
println(ptdfs)
println("Used capacities over AB, AC, BC lines")
println(value.(prob.model[:FlowBasedAB]), value.(prob.model[:FlowBasedAC]), value.(prob.model[:FlowBasedBC]))

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.88000e+00
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 2.88000e+00
  Primal solution :
    FlowAB[1] : 2.40000e+00
    FlowAC[1] : 2.40000e-01
    FlowBA[1] : 0.00000e+00
    FlowBC[1] : 0.00000e+00
    FlowCA[1] : 0.00000e+00
    FlowCB[1] : 0.00000e+00
    FlowPowerA[1] : 2.88000e+00
    FlowPowerB[1] : 0.00000e+00
    FlowPowerC[1] : 0.00000e+00
  Dual solution :
    BalanceA[1] : -1.00000e+00
    BalanceB[1] : -1.00000e+00
    BalanceC[1] : -1.00000e+00
    FlowBasedAB[1] : 0.00000e+00
    FlowBasedAC[1] : 0.00000e+00
    FlowBasedBC[1] : 0.00000e+00

* Work counters
  Solve time (sec)   : 8.20160e-05
  Simplex iterations : 4
  Barrier iterations : 0
  Node count         